In [1]:
# !pip install openai

In [2]:
# !pip install --upgrade openai

In [ ]:
import os
import openai
import time
import pandas as pd

In [2]:
openai.organization = ""#"X"
openai.api_key = "X"
openai.Model.list()

AuthenticationError: Incorrect API key provided: X. You can find your API key at https://platform.openai.com/account/api-keys.

In [17]:
template = lambda q, a: f"""<P {q} /P>. <R {a} /R>. Pensemos paso a paso."""
q = "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras."
a = "estamal por caben 20 dados"
prompt = template(q, a)
prompt

'<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>. <R estamal por caben 20 dados /R>. Pensemos paso a paso.'

In [18]:
def chatgpt(message, text, temp=0.7, max_t=5):
    try: 
        output = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": message},
                  {"role": "user", "content": text},
            ],
            temperature=temp,
            max_tokens=max_t,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    except:
        print("Thinking...")
        time.sleep(1)
        return chatgpt(message, text, temp=temp, max_t=max_t)
    return output

In [20]:
%%time
system = "Usted es un clasificador de incoherencia en las respuestas a preguntas abiertas de estudiantes de matemáticas de cuarto grado. Le daré una pregunta y respuestas diferentes, y luego tendrá que decir si la respuesta es coherente o incoherente a la pregunta. Marcaremos las preguntas entre <P /P> y las respuestas entre <R /R>. Entregará la etiqueta incoherente o coherente entre <E /E>. Las respuestas incorrectas aún pueden ser coherentes, dado que la incoherencia significa que la respuesta no tiene sentido con lo que se pregunta y no necesariamente con su Correctitud."
chatgpt(system, prompt, temp=0.7, max_t=10)["choices"][0]["message"]["content"]

Wall time: 777 ms


'<E incoherente /E>. La respuesta'

In [22]:
test_set = pd.read_excel("data/test_task_C1.xlsx", index_col=0)
test_set

,Q_id,Q,A,label
id,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1
...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1


In [23]:
q, a, label = test_set.sample(1).iloc[0]["Q A label".split()]
prompt = template(q, a)
prompt, label

('<P ¿ Cual es el valor posicional del dígito 4 en el numero 864? /P>. <R restar o sumar\nsite resulta muy bien\n /R>. Pensemos paso a paso.',
 1)

In [26]:
chatgpt(system, prompt, temp=0.7, max_t=10)["choices"][0]["message"]["content"]

'<E incoherente /E> La respuesta'

In [28]:
%%time
responses = []
for i, ix in enumerate(test_set.index):
    q, a, label = test_set.loc[ix]["Q A label".split()]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(system, template(q, a), temp=0.7, max_t=10)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/test_set.shape[0], ix, o, label)

0.14771048744460857 85815 La respuesta es incoherente porque no res 0
0.29542097488921715 64812 E/E. Aunque la respuesta menciona el 0
0.4431314623338257 84148 <E coherente /E>. La respuesta es 0
0.5908419497784343 88565 La respuesta es incoherente, ya que 1
0.7385524372230429 51688 <E Coherente /E> 1
0.8862629246676514 73857 <E incoherente /E> La respuesta 1
1.03397341211226 68843 <E incoherente /E>. La respuesta 0
1.1816838995568686 73747 <E incoherente /E>. La respuesta 0
1.329394387001477 92561 <E incoherente /E>. La respuesta 1
1.4771048744460857 90379 <E incoherente /E>. La respuesta 1
1.6248153618906942 84122 La respuesta es incoherente a la pregunta 0
1.7725258493353029 75775 <E incoherente /E>. La respuesta 0
1.9202363367799113 56351 La respuesta dada es incoherente con 0
2.06794682422452 85614 La respuesta dada es incoherente con 0
2.2156573116691285 83090 <E coherente /E>. La respuesta es 0
2.363367799113737 75196 La respuesta dada no es coherente con 0
2.511078286558346 658

Thinking...


Thinking...


Thinking...


Thinking...


58.493353028064995 60129 <E incoherente /E> 1
58.6410635155096 83581 La respuesta es incoherente con la pregunta 0
58.78877400295421 84099 <E incoherente /E> La respuesta 0
58.93648449039882 90253 <E incoherente /E>. La respuesta 0
59.084194977843424 49223 <E incoherente /E> 0
59.23190546528804 74355 <E Incoherente /E>. La respuesta 0
59.37961595273264 64328 La respuesta dada es coherente con la 0
59.527326440177255 60126 <E coherente /E>. La respuesta es 0
59.67503692762186 73801 <E incoherente /E> La respuesta 1
59.822747415066466 67152 <E incoherente. La respuesta no está 0
59.97045790251108 89551 <E incoherente /E>. La respuesta 0
60.118168389955684 90382 <E incoherente /E>. La respuesta 0
60.2658788774003 74487 E (incoherente)  La respuesta d 0
60.4135893648449 75198 <E incoherente /E>. La respuesta 0
60.561299852289515 65072 <E coherente /E>. La respuesta es 0
60.70901033973412 64002 <E incoherente /E> La respuesta 1
60.85672082717873 89356 <E La respuesta dada es incoherente 0
6

In [29]:
test_c1_pred = test_set.copy()
test_c1_pred["chatgpt"] = [x[1] for x in responses]
test_c1_pred

,Q_id,Q,A,label,chatgpt
id,,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,La respuesta es incoherente porque no res
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,E/E. Aunque la respuesta menciona el
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,<E coherente /E>. La respuesta es
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,"La respuesta es incoherente, ya que"
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,<E Coherente /E>
...,...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,<E incoherente /E> La respuesta
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,<E incoherente /E>. La respuesta
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,La pregunta podría ser: ¿Cuántos estudiantes


In [30]:
test_c1_pred.to_excel("chatgpt_responses_test_c1_pred_cot_v3.xlsx")

In [31]:
from sklearn.metrics import classification_report

In [33]:
print(classification_report(test_c1_pred["label"], test_c1_pred["chatgpt"].apply(lambda x: int("incoh" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.8447    0.2514    0.3875       541
           1     0.2151    0.8162    0.3405       136

    accuracy                         0.3648       677
   macro avg     0.5299    0.5338    0.3640       677
weighted avg     0.7182    0.3648    0.3780       677



In [34]:
root = "D:\\github\\coherence-prediction\\"
test_label_Q = pd.read_excel(root+"gpt_label_v7_comp.xlsx").set_index("id")
test_c1_pred["label_Q"] = test_label_Q.loc[test_c1_pred.index]["tipo_preg"]
print(classification_report(test_c1_pred[test_c1_pred["label_Q"] == 3]["label"], test_c1_pred[test_c1_pred["label_Q"] == 3]["chatgpt"].apply(lambda x: int("incoherente" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.9014    0.2443    0.3844       262
           1     0.1466    0.8293    0.2491        41

    accuracy                         0.3234       303
   macro avg     0.5240    0.5368    0.3167       303
weighted avg     0.7993    0.3234    0.3661       303

